In [ ]:
class GPUMonitor:
    """GPU使用率とメモリ使用量をモニタリング"""
    
    def __init__(self, interval=0.1):
        self.interval = interval
        self.gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        self.monitoring = False
        self.timestamps = deque(maxlen=10000)
        self.gpu_utils = deque(maxlen=10000)
        self.mem_utils = deque(maxlen=10000)
        self.mem_used = deque(maxlen=10000)
        self.thread = None
    
    def _monitor_loop(self):
        """モニタリングループ"""
        start_time = time.time()
        while self.monitoring:
            current_time = time.time() - start_time
            
            # GPU使用率
            util = pynvml.nvmlDeviceGetUtilizationRates(self.gpu_handle)
            self.gpu_utils.append(util.gpu)
            
            # メモリ使用量
            mem_info = pynvml.nvmlDeviceGetMemoryInfo(self.gpu_handle)
            self.mem_utils.append(100 * mem_info.used / mem_info.total)
            self.mem_used.append(mem_info.used / (1024**3))  # GB
            
            self.timestamps.append(current_time)
            time.sleep(self.interval)
    
    def start(self):
        """モニタリング開始"""
        self.monitoring = True
        self.thread = threading.Thread(target=self._monitor_loop)
        self.thread.start()
    
    def stop(self):
        """モニタリング停止"""
        self.monitoring = False
        if self.thread:
            self.thread.join()
    
    def get_data(self):
        """データ取得"""
        return {
            'timestamps': list(self.timestamps),
            'gpu_utils': list(self.gpu_utils),
            'mem_utils': list(self.mem_utils),
            'mem_used_gb': list(self.mem_used)
        }
    
    def plot(self, title="GPU Monitoring Results"):
        """結果のプロット"""
        data = self.get_data()
        if not data['timestamps']:
            print("No monitoring data available")
            return
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # GPU使用率
        ax1.plot(data['timestamps'], data['gpu_utils'], 'b-', linewidth=1.5)
        ax1.set_ylabel('GPU Utilization (%)')
        ax1.set_ylim(0, 105)
        ax1.grid(True, alpha=0.3)
        ax1.set_title(f"{title} - GPU Utilization")
        
        # メモリ使用量
        ax2.plot(data['timestamps'], data['mem_used_gb'], 'r-', linewidth=1.5, label='Used')
        ax2.axhline(y=24, color='k', linestyle='--', alpha=0.5, label='Total (24GB)')
        ax2.set_xlabel('Time (seconds)')
        ax2.set_ylabel('GPU Memory (GB)')
        ax2.set_ylim(0, 26)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        ax2.set_title("GPU Memory Usage")
        
        plt.tight_layout()
        plt.show()
        
        # 統計情報
        print(f"\nGPU Utilization Statistics:")
        print(f"  Average: {np.mean(data['gpu_utils']):.1f}%")
        print(f"  Maximum: {np.max(data['gpu_utils']):.1f}%")
        print(f"  Minimum: {np.min(data['gpu_utils']):.1f}%")
        
        print(f"\nGPU Memory Statistics:")
        print(f"  Average: {np.mean(data['mem_used_gb']):.2f} GB")
        print(f"  Maximum: {np.max(data['mem_used_gb']):.2f} GB")
        print(f"  Minimum: {np.min(data['mem_used_gb']):.2f} GB")

# Helper functions
def print_rmm_statistics():
    """RMM統計情報を表示"""
    stats = rmm.statistics.get_statistics()
    print("\nRMM Memory Statistics:")
    print(f"  Current allocated: {stats.current_bytes / (1024**3):.2f} GB")
    print(f"  Peak allocated: {stats.peak_bytes / (1024**3):.2f} GB")
    print(f"  Total allocations: {stats.n_allocations}")
    print(f"  Total deallocations: {stats.n_deallocations}")

def print_cupy_memory_info():
    """CuPyメモリプール情報を表示"""
    mempool = cp.get_default_memory_pool()
    print("\nCuPy Memory Pool:")
    print(f"  Used: {mempool.used_bytes() / (1024**3):.2f} GB")
    print(f"  Total: {mempool.total_bytes() / (1024**3):.2f} GB")

# モニターのインスタンス化
monitor = GPUMonitor(interval=0.05)
print("GPU Monitor initialized")

class GPUMonitor:
    """GPU使用率とメモリ使用量をモニタリング"""
    
    def __init__(self, interval=0.1):
        self.interval = interval
        self.gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        self.monitoring = False
        self.timestamps = deque(maxlen=10000)
        self.gpu_utils = deque(maxlen=10000)
        self.mem_utils = deque(maxlen=10000)
        self.mem_used = deque(maxlen=10000)
        self.thread = None
    
    def _monitor_loop(self):
        """モニタリングループ"""
        start_time = time.time()
        while self.monitoring:
            current_time = time.time() - start_time
            
            # GPU使用率
            util = pynvml.nvmlDeviceGetUtilizationRates(self.gpu_handle)
            self.gpu_utils.append(util.gpu)
            
            # メモリ使用量
            mem_info = pynvml.nvmlDeviceGetMemoryInfo(self.gpu_handle)
            self.mem_utils.append(100 * mem_info.used / mem_info.total)
            self.mem_used.append(mem_info.used / (1024**3))  # GB
            
            self.timestamps.append(current_time)
            time.sleep(self.interval)
    
    def start(self):
        """モニタリング開始"""
        self.monitoring = True
        self.thread = threading.Thread(target=self._monitor_loop)
        self.thread.start()
    
    def stop(self):
        """モニタリング停止"""
        self.monitoring = False
        if self.thread:
            self.thread.join()
    
    def get_data(self):
        """データ取得"""
        return {
            'timestamps': list(self.timestamps),
            'gpu_utils': list(self.gpu_utils),
            'mem_utils': list(self.mem_utils),
            'mem_used_gb': list(self.mem_used)
        }
    
    def plot(self, title="GPU Monitoring Results"):
        """結果のプロット"""
        data = self.get_data()
        if not data['timestamps']:
            print("No monitoring data available")
            return
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # GPU使用率
        ax1.plot(data['timestamps'], data['gpu_utils'], 'b-', linewidth=1.5)
        ax1.set_ylabel('GPU Utilization (%)')
        ax1.set_ylim(0, 105)
        ax1.grid(True, alpha=0.3)
        ax1.set_title(f"{title} - GPU Utilization")
        
        # メモリ使用量
        ax2.plot(data['timestamps'], data['mem_used_gb'], 'r-', linewidth=1.5, label='Used')
        ax2.axhline(y=24, color='k', linestyle='--', alpha=0.5, label='Total (24GB)')
        ax2.set_xlabel('Time (seconds)')
        ax2.set_ylabel('GPU Memory (GB)')
        ax2.set_ylim(0, 26)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        ax2.set_title("GPU Memory Usage")
        
        plt.tight_layout()
        plt.show()
        
        # 統計情報
        print(f"\nGPU Utilization Statistics:")
        print(f"  Average: {np.mean(data['gpu_utils']):.1f}%")
        print(f"  Maximum: {np.max(data['gpu_utils']):.1f}%")
        print(f"  Minimum: {np.min(data['gpu_utils']):.1f}%")
        
        print(f"\nGPU Memory Statistics:")
        print(f"  Average: {np.mean(data['mem_used_gb']):.2f} GB")
        print(f"  Maximum: {np.max(data['mem_used_gb']):.2f} GB")
        print(f"  Minimum: {np.min(data['mem_used_gb']):.2f} GB")

# Helper functions
def print_rmm_statistics():
    """RMM統計情報を表示"""
    try:
        stats = rmm.statistics.get_statistics()
        if stats:
            print("\nRMM Memory Statistics:")
            print(f"  Current allocated: {stats.current_bytes / (1024**3):.2f} GB")
            print(f"  Peak allocated: {stats.peak_bytes / (1024**3):.2f} GB")
            print(f"  Total allocations: {stats.n_allocations}")
            print(f"  Total deallocations: {stats.n_deallocations}")
        else:
            print("\nRMM Memory Statistics: No statistics available")
    except Exception as e:
        print(f"\nRMM Memory Statistics: Error - {e}")

def print_cupy_memory_info():
    """CuPyメモリプール情報を表示"""
    mempool = cp.get_default_memory_pool()
    print("\nCuPy Memory Pool:")
    print(f"  Used: {mempool.used_bytes() / (1024**3):.2f} GB")
    print(f"  Total: {mempool.total_bytes() / (1024**3):.2f} GB")

# モニターのインスタンス化
monitor = GPUMonitor(interval=0.05)
print("GPU Monitor initialized")

In [ ]:
# GPU monitoring tools
try:
    import pynvml
except ImportError:
    !pip install nvidia-ml-py3
    import pynvml

import cupy as cp
import rmm
from rmm.statistics import ProfilerRecords, statistics
import threading
from collections import deque
import gc
import psutil

# Initialize NVML for GPU monitoring
pynvml.nvmlInit()
gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# Get GPU info
gpu_name = pynvml.nvmlDeviceGetName(gpu_handle).decode('utf-8')
gpu_mem_info = pynvml.nvmlDeviceGetMemoryInfo(gpu_handle)
print(f"GPU: {gpu_name}")
print(f"GPU Memory: {gpu_mem_info.total / (1024**3):.1f} GB")

# RMM statistics を有効化
rmm.statistics.enable_statistics()

# RMM を再初期化（統計機能付き）
rmm.reinitialize(
    managed_memory=False,  # まずは通常のGPUメモリで試す
    pool_allocator=True,
    initial_pool_size=2<<30,    # 2GB
    maximum_pool_size=22<<30,   # 22GB (24GBの GPU用)
    logging=True
)

print("\nRMM memory profiling enabled")

# GPU monitoring tools
try:
    import pynvml
except ImportError:
    !pip install nvidia-ml-py3
    import pynvml

import cupy as cp
import rmm
from rmm.statistics import ProfilerRecords, statistics
import threading
from collections import deque
import gc
import psutil

# Initialize NVML for GPU monitoring
pynvml.nvmlInit()
gpu_handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# Get GPU info
gpu_name_raw = pynvml.nvmlDeviceGetName(gpu_handle)
gpu_name = gpu_name_raw.decode('utf-8') if isinstance(gpu_name_raw, bytes) else gpu_name_raw
gpu_mem_info = pynvml.nvmlDeviceGetMemoryInfo(gpu_handle)
print(f"GPU: {gpu_name}")
print(f"GPU Memory: {gpu_mem_info.total / (1024**3):.1f} GB")

# RMM statistics を有効化
rmm.statistics.enable_statistics()

# RMM を再初期化（統計機能付き）
rmm.reinitialize(
    managed_memory=False,  # まずは通常のGPUメモリで試す
    pool_allocator=True,
    initial_pool_size=2<<30,    # 2GB
    maximum_pool_size=22<<30,   # 22GB (24GBの GPU用)
    logging=True
)

print("\nRMM memory profiling enabled")

## 1. 環境設定とインポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pathlib import Path
import sys

# Add parent directory to path for imports
sys.path.append('..')

# Import decoders
from gpu_can_decoder import GPUCANDecoder
from cpu_can_decoder import CPUCANDecoder

print("ライブラリのインポート完了")

In [ ]:
def generate_synthetic_can_data(n_messages):
    """合成CANデータの生成（OpenPilot DBCファイルに準拠）"""
    # リアルなCANデータ分布を模倣
    address_distribution = {
        170: 0.037,  # 4輪速度
        37: 0.037,   # ステアリング
        36: 0.037,
        740: 0.044,
        608: 0.022,
        180: 0.018,
    }
    
    # アドレスを生成
    addresses = []
    for addr, prob in address_distribution.items():
        count = int(n_messages * prob)
        addresses.extend([addr] * count)
    
    # 残りはランダムなアドレス
    remaining = n_messages - len(addresses)
    other_addresses = np.random.choice([452, 466, 467, 705, 321, 562], remaining)
    addresses.extend(other_addresses)
    
    # シャッフル
    np.random.shuffle(addresses)
    addresses = np.array(addresses[:n_messages], dtype=np.int64)
    
    # タイムスタンプ（実データと同じ範囲）
    timestamps = np.linspace(46408.0, 46468.0, n_messages)
    
    # データバイト
    data_bytes = np.zeros((n_messages, 8), dtype=np.uint8)
    
    for i in range(n_messages):
        if addresses[i] == 170:  # 4輪速度
            # OpenPilot DBC: (0.01,-67.67) "kph" for Toyota RAV4
            for j in range(4):
                speed_kmh = np.random.uniform(55, 65)  # 55-65 km/h
                raw_value = int((speed_kmh + 67.67) / 0.01)
                data_bytes[i, j*2] = (raw_value >> 8) & 0xFF
                data_bytes[i, j*2 + 1] = raw_value & 0xFF
        elif addresses[i] == 37:  # ステアリング
            # 固定値パターン（実データと同じ）
            data_bytes[i] = [0x00, 0x00, 0x10, 0x00, 0xC0, 0x00, 0x00, 0xFD]
        else:
            # その他はランダム
            data_bytes[i] = np.random.randint(0, 256, 8, dtype=np.uint8)
    
    return timestamps, addresses, data_bytes

In [ ]:
# 生CANデータの構造を確認
print("=== 生CANデータの列指向配列レイアウト ===\n")

# サンプルデータを生成して構造を表示
n_sample = 5
sample_timestamps, sample_addresses, sample_data_bytes = generate_synthetic_can_data(n_sample)

print("生CANデータは3つの列指向配列で構成されています：\n")

print("1. タイムスタンプ配列 (timestamps):")
print(f"   - 型: {sample_timestamps.dtype}")
print(f"   - 形状: {sample_timestamps.shape}")
print(f"   - 例: {sample_timestamps}")

print("\n2. アドレス配列 (addresses):")
print(f"   - 型: {sample_addresses.dtype}")
print(f"   - 形状: {sample_addresses.shape}")
print(f"   - 例: {sample_addresses}")

print("\n3. データバイト配列 (data_bytes):")
print(f"   - 型: {sample_data_bytes.dtype}")
print(f"   - 形状: {sample_data_bytes.shape}")
print(f"   - 例:")
for i in range(n_sample):
    print(f"     メッセージ{i}: {sample_data_bytes[i]}")

print("\n=== データレイアウトの特徴 ===")
print("- 列指向: 各属性（timestamp, address, data）が個別の配列として格納")
print("- 効率的: GPUでの並列処理に適した連続メモリレイアウト")
print("- Apache Arrow互換: 列指向フォーマットへの変換が容易")

# メモリレイアウトの可視化
import matplotlib.patches as mpatches

fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# 各配列のメモリ表現
n_messages = 10
bar_height = 0.8

# タイムスタンプ配列
for i in range(n_messages):
    rect = mpatches.Rectangle((i, 0), 0.9, bar_height, 
                            facecolor='lightblue', edgecolor='black', linewidth=0.5)
    ax.add_patch(rect)
    ax.text(i + 0.45, 0.4, f't{i}', ha='center', va='center', fontsize=8)

# アドレス配列
for i in range(n_messages):
    rect = mpatches.Rectangle((i, 1.2), 0.9, bar_height, 
                            facecolor='lightgreen', edgecolor='black', linewidth=0.5)
    ax.add_patch(rect)
    ax.text(i + 0.45, 1.6, f'a{i}', ha='center', va='center', fontsize=8)

# データバイト配列（2次元）
for i in range(n_messages):
    for j in range(8):
        rect = mpatches.Rectangle((i, 2.4 + j * 0.1), 0.9, 0.08, 
                                facecolor='lightcoral', edgecolor='black', linewidth=0.5)
        ax.add_patch(rect)

ax.text(-1.5, 0.4, 'timestamps[]', ha='right', va='center', fontweight='bold')
ax.text(-1.5, 1.6, 'addresses[]', ha='right', va='center', fontweight='bold')
ax.text(-1.5, 2.8, 'data_bytes[][]', ha='right', va='center', fontweight='bold')

ax.set_xlim(-2, n_messages)
ax.set_ylim(-0.5, 3.5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('生CANデータの列指向配列レイアウト', fontsize=14, fontweight='bold', pad=20)

# 矢印で各メッセージの関係を示す
for i in [3, 7]:  # 例として2つのメッセージ
    ax.annotate('', xy=(i + 0.45, 2.3), xytext=(i + 0.45, 0.9),
                arrowprops=dict(arrowstyle='<->', color='gray', lw=0.5))
    ax.text(i + 0.7, 1.5, f'メッセージ{i}', ha='left', va='center', fontsize=7, color='gray')

plt.tight_layout()
plt.show()

# 実際のデータサイズ計算
print("\n=== メモリ使用量の例 ===")
for n in [10_000, 100_000, 1_000_000, 10_000_000]:
    timestamp_size = n * 8  # float64
    address_size = n * 8    # int64
    data_size = n * 8 * 1   # uint8 * 8
    total_size = timestamp_size + address_size + data_size
    print(f"{n:>10,} メッセージ: {total_size / (1024**2):>8.1f} MB")
    print(f"            - timestamps: {timestamp_size / (1024**2):>6.1f} MB")
    print(f"            - addresses:  {address_size / (1024**2):>6.1f} MB")
    print(f"            - data_bytes: {data_size / (1024**2):>6.1f} MB")

In [ ]:
# テスト用データの生成 - 10,000,000メッセージまで拡張
test_sizes = [
    10_000,        # 10K
    50_000,        # 50K
    100_000,       # 100K
    500_000,       # 500K
    1_000_000,     # 1M
    5_000_000,     # 5M
    10_000_000,    # 10M
]
print("テストデータサイズ:", test_sizes)

## 1.1 生CANデータの列指向配列レイアウト

In [ ]:
print("=== Initial Memory State ===")
print_rmm_statistics()
print_cupy_memory_info()

# GPU info from NVML
mem_info = pynvml.nvmlDeviceGetMemoryInfo(gpu_handle)
print(f"\nGPU Memory (NVML):")
print(f"  Total: {mem_info.total / (1024**3):.1f} GB")
print(f"  Used: {mem_info.used / (1024**3):.2f} GB")
print(f"  Free: {mem_info.free / (1024**3):.2f} GB")

# RAM info
print(f"\nSystem RAM:")
print(f"  Available: {psutil.virtual_memory().available / (1024**3):.1f} GB")
print(f"  Total: {psutil.virtual_memory().total / (1024**3):.1f} GB")

## 2.1 初期メモリ状態の確認

## 1.1 GPU プロファイリング設定

In [ ]:
# デコーダーの初期化
gpu_decoder = GPUCANDecoder(batch_size=500_000)
cpu_decoder = CPUCANDecoder(batch_size=100_000)

# ベンチマーク結果格納
benchmark_results = []

for n_messages in test_sizes:
    print(f"\n--- {n_messages:,} メッセージの処理 ---")
    
    # メモリクリア
    gc.collect()
    cp.get_default_memory_pool().free_all_blocks()
    
    # データ生成
    timestamps, addresses, data_bytes = generate_synthetic_can_data(n_messages)
    data_size_mb = (timestamps.nbytes + addresses.nbytes + data_bytes.nbytes) / (1024**2)
    print(f"データサイズ: {data_size_mb:.1f} MB")
    
    # 初期メモリ状態
    print("Pre-processing memory:")
    print_rmm_statistics()
    print_cupy_memory_info()
    
    # GPU処理（モニタリング付き）
    monitor.start()
    
    gpu_start = time.time()
    with statistics.profiler(name=f"GPU_decode_{n_messages}"):
        gpu_results = gpu_decoder.decode_batch(timestamps, addresses, data_bytes)
        cp.cuda.Stream.null.synchronize()  # GPU同期
    gpu_time = time.time() - gpu_start
    
    monitor.stop()
    
    # CPU処理（大きいデータは時間がかかるため制限）
    if n_messages <= 100_000:
        cpu_start = time.time()
        cpu_results = cpu_decoder.decode_batch(timestamps, addresses, data_bytes)
        cpu_time = time.time() - cpu_start
    else:
        # 線形推定
        cpu_time = benchmark_results[-1]['cpu_time'] * (n_messages / benchmark_results[-1]['n_messages'])
    
    # モニタリング結果の取得
    monitor_data = monitor.get_data()
    
    # 結果記録
    result = {
        'n_messages': n_messages,
        'data_size_mb': data_size_mb,
        'gpu_time': gpu_time,
        'cpu_time': cpu_time,
        'speedup': cpu_time / gpu_time,
        'gpu_throughput': n_messages / gpu_time / 1e6,
        'cpu_throughput': n_messages / cpu_time / 1e6,
        'gpu_util_avg': np.mean(monitor_data['gpu_utils']) if monitor_data['gpu_utils'] else 0,
        'gpu_util_max': np.max(monitor_data['gpu_utils']) if monitor_data['gpu_utils'] else 0,
        'mem_used_avg_gb': np.mean(monitor_data['mem_used_gb']) if monitor_data['mem_used_gb'] else 0,
        'mem_used_max_gb': np.max(monitor_data['mem_used_gb']) if monitor_data['mem_used_gb'] else 0
    }
    benchmark_results.append(result)
    
    print(f"GPU処理時間: {gpu_time:.4f}秒 ({result['gpu_throughput']:.1f} Mmsg/s)")
    print(f"CPU処理時間: {cpu_time:.4f}秒 ({result['cpu_throughput']:.1f} Mmsg/s)")
    print(f"高速化率: {result['speedup']:.1f}x")
    print(f"GPU使用率: 平均{result['gpu_util_avg']:.1f}%, 最大{result['gpu_util_max']:.1f}%")
    print(f"GPUメモリ: 平均{result['mem_used_avg_gb']:.2f}GB, 最大{result['mem_used_max_gb']:.2f}GB")
    
    # GPU使用率のプロット
    monitor.plot(title=f"GPU Processing - {n_messages:,} messages")
    
    # Post-processing memory state
    print("Post-processing memory:")
    print_rmm_statistics()
    print_cupy_memory_info()

# DataFrameに変換
benchmark_df = pd.DataFrame(benchmark_results)
benchmark_df

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Processing time comparison
ax1.plot(benchmark_df['n_messages'], benchmark_df['gpu_time'], 'b-o', label='GPU', linewidth=2, markersize=8)
ax1.plot(benchmark_df['n_messages'], benchmark_df['cpu_time'], 'r-o', label='CPU', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Messages')
ax1.set_ylabel('Processing Time (seconds)')
ax1.set_title('Processing Time Comparison')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Speedup ratio
ax2.plot(benchmark_df['n_messages'], benchmark_df['speedup'], 'g-o', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Messages')
ax2.set_ylabel('Speedup (times)')
ax2.set_title('GPU Speedup Ratio')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=1, color='k', linestyle='--', alpha=0.5)

# GPU使用率 vs データサイズ
ax3.plot(benchmark_df['n_messages'], benchmark_df['gpu_util_avg'], 'b-o', 
         label='Average', linewidth=2, markersize=8)
ax3.plot(benchmark_df['n_messages'], benchmark_df['gpu_util_max'], 'r--o', 
         label='Maximum', linewidth=2, markersize=8)
ax3.set_xlabel('Number of Messages')
ax3.set_ylabel('GPU Utilization (%)')
ax3.set_title('GPU Utilization vs Data Size')
ax3.set_xscale('log')
ax3.legend()
ax3.grid(True, alpha=0.3)

# メモリ使用量 vs データサイズ
ax4.plot(benchmark_df['n_messages'], benchmark_df['mem_used_avg_gb'], 'g-o', 
         label='Average', linewidth=2, markersize=8)
ax4.plot(benchmark_df['n_messages'], benchmark_df['mem_used_max_gb'], 'm--o', 
         label='Maximum', linewidth=2, markersize=8)
ax4.set_xlabel('Number of Messages')
ax4.set_ylabel('GPU Memory (GB)')
ax4.set_title('GPU Memory Usage vs Data Size')
ax4.set_xscale('log')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Results summary
print("\n=== Benchmark Results Summary ===")
print(f"Maximum speedup: {benchmark_df['speedup'].max():.1f}x")
print(f"Maximum GPU throughput: {benchmark_df['gpu_throughput'].max():.1f} Mmessages/sec")
print(f"Average CPU throughput: {benchmark_df['cpu_throughput'].mean():.2f} Mmessages/sec")
print(f"Average GPU utilization: {benchmark_df['gpu_util_avg'].mean():.1f}%")
print(f"Peak GPU memory usage: {benchmark_df['mem_used_max_gb'].max():.2f} GB")